# Solving the program without $K_{d+1}$


Primal program:
- variable $x$ where $x_L$ represents the probability of getting local view $L$ and $x_L\ge 0$ for all $L$
- objective is $\min \sum_L x_L\alpha_L$ where $\alpha_L$ is the probability $u$ gets $+$ in local view $L$
- let $\gamma^{u}(j)_L = \Pr(|\{ v\in N(u) : \sigma_v=+\}| = j \mid L)$
- let $\gamma^{N(u)}(j)_L = \frac{1}{d}\sum_{v\in N(u)}\Pr(|\{ w\in N(v) : \sigma_w=+\}| = j \mid L)$
- probability constraint: $\sum_L x_L \ge 1$ (equality holds but we seem to get by with inequality)
- gamma constraints: for $0\le j \le d-2$ we have $\sum_L\left(\gamma^u(j)_L-\gamma^{N(u)}(j)_L\right)x_L \ge 0$ (the $j=d$ case follows from the others and $j=d-1$ appears to be redundant, also equality holds but inequality seems sufficient)

Dual program:
- variables are $y_p$ for the probability constraint, $y_0,\dotsc, y_{d-2}$ for the $\gamma(j)$ constraints (all are nonnegative, perhaps?)
- dual objective is $\max y_p$
- dual constraint for each $L$: $y_p + \sum_{j=0}^{d-2}\left(\gamma^u(j)_L-\gamma^{N(u)}(j)_L\right)y_j\le \alpha_L$
- solve for the $d$ dual variables in the tight constraints we find empirically:
    - $d=3$: [15, 20, 21]
    - $d=4$: [198, 206, 216, 228]
- now we have values for $y_p, y_0, \dotsc, y_{d-2}$ we have to prove that they are feasible:
    - each variable is nonnegative
    - for all $L$, dual constraint holds

Each of these two final tasks is equivalent to showing that some giant polynomial involving $B$ and $\lambda$ is nonnegative, and that is difficult in general. Given suitable bounds on the parameters, e.g. that $0\le B\le 0.9*(d-2)/d$ and $0\le \lambda \le B^2$, what we can do is try to make variable substitutions so that the resulting polynomials are of nonnegative variables with nonnegative coefficients.

For examples on how this can work, see e.g.
- https://arxiv.org/pdf/1610.08496
- https://arxiv.org/pdf/1611.01474
- https://arxiv.org/pdf/1801.07547

Usually, the first task is to figure out the denominators in the relevant inequalities and multiply by them to clear denominators.

In [2]:
import os.path

from collections import namedtuple
from itertools import product

from sage.all import diff, graphs, ln, load, Rational, save, srange, sqrt, var
from sage.numerical.mip import MixedIntegerLinearProgram

from tqdm.autonotebook import tqdm

LData = namedtuple("LData", ["Ls", "version"])
DATA_VERSION = 0
def default_filename(d):
    return f"data/ising_d{d}.sobj"

def load_data(d, filename=None):
    if filename is None:
        filename = default_filename(d)

    data = load(filename)
    return data

def lc(d, B):
    r = var("r")
    s = var("s")
    bc = var("bc", latex_name="B_c")

    ret = (
        (1 - sqrt(r / s))
        * ((1 + sqrt(r * s)) / (1 - sqrt(r * s))) ** ((1 + bc) / (-1 + bc))
    ) / (1 + sqrt(r / s))

    return ret.subs(r=(bc - B) / (bc + B), s=(1 - B) / (1 + B)).subs(bc=(d - 2) / d)


In [5]:
d = 4
Ls = load_data(d).Ls[:-1] # no K_{d+1} please
if d == 3:
    tight = [15, 20, 21]
elif d == 4:
    tight = [198, 206, 216, 228]
else:
    raise ValueError(f"no plan for d={d}")

Atranspose = matrix([
    [1] + [Ls[t]["gu"][j] - Ls[t]["gNu"][j] for j in range(d-1)] for t in tight
])
c = vector(Ls[t]["pu"] for t in tight)
ys = Atranspose.solve_right(c)

Bsub = 25/100
lsub = Rational(lc(d, Bsub).n(digits=4))

print(Bsub, lsub)
# TODO: prove for all Ls, ys[0] + sum(ys[j] * (L["gu"][j] - L["gNu"][j]) for j in range(d-1)) <= L["pu"]
for i, L in enumerate(tqdm(Ls)):
    ineq = (ys[0] + sum(ys[j+1] * (L["gu"][j] - L["gNu"][j]) for j in range(d-1)) <= L["pu"])
    # print(ineq)
    if not((ys[0] + sum(ys[j+1] * (L["gu"][j] - L["gNu"][j]) for j in range(d-1)) <= L["pu"]).subs(B=Bsub,l=lsub)):
        print(f"counterexample found at index {i}")

1/4 75/9224


  0%|          | 0/229 [00:00<?, ?it/s]